Reference: [Iafoss starter](https://www.kaggle.com/code/iafoss/rna-starter-0-186-lb/notebook)

## Download data

download kaggle.json from kaggle settings

In [2]:
# !pip install kaggle
# !mkdir ~/.kaggle
# !cp kaggle.json ~/.kaggle/kaggle.json
# !chmod 600 ~/.kaggle/kaggle.json
# kaggle datasets download iafoss/stanford-ribonanza-rna-folding-converted
# unzip stanford-ribonanza-rna-folding-converted.zip
# !kaggle kernels output iafoss/rna-starter-0-186-lb

## Setup

In [1]:
import pandas as pd
import os, gc
import numpy as np
from sklearn.model_selection import KFold
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from fastai.vision.all import *
from torch.cuda.amp import GradScaler, autocast

In [2]:
def flatten(o):
    "Concatenate all collections and items as a generator"
    for item in o:
        if isinstance(o, dict): yield o[item]; continue
        elif isinstance(item, str): yield item; continue
        try: yield from flatten(item)
        except TypeError: yield item


@delegates(GradScaler)
class MixedPrecision(Callback):
    "Mixed precision training using Pytorch's `autocast` and `GradScaler`"
    order = 10
    def __init__(self, **kwargs): self.kwargs = kwargs
    def before_fit(self): 
        self.autocast,self.learn.scaler,self.scales = autocast(),GradScaler(**self.kwargs),L()
    def before_batch(self): self.autocast.__enter__()
    def after_pred(self):
        if next(flatten(self.pred)).dtype==torch.float16: self.learn.pred = to_float(self.pred)
    def after_loss(self): self.autocast.__exit__(None, None, None)
    def before_backward(self): self.learn.loss_grad = self.scaler.scale(self.loss_grad)
    def before_step(self):
        "Use `self` as a fake optimizer. `self.skipped` will be set to True `after_step` if gradients overflow. "
        self.skipped=True
        self.scaler.step(self)
        if self.skipped: raise CancelStepException()
        self.scales.append(self.scaler.get_scale())
    def after_step(self): self.learn.scaler.update()

    @property 
    def param_groups(self): 
        "Pretend to be an optimizer for `GradScaler`"
        return self.opt.param_groups
    def step(self, *args, **kwargs): 
        "Fake optimizer step to detect whether this batch was skipped from `GradScaler`"
        self.skipped=False
    def after_fit(self): self.autocast,self.learn.scaler,self.scales = None,None,None
        

In [3]:
import fastai
fastai.callback.fp16.MixedPrecision = MixedPrecision

In [4]:
class LenMatchBatchSampler(torch.utils.data.BatchSampler):
    def __iter__(self):
        buckets = [[]] * 100
        yielded = 0

        for idx in self.sampler:
            s = self.sampler.data_source[idx]
            if isinstance(s,tuple): L = s[0]["mask"].sum()
            else: L = s["mask"].sum()
            L = max(1,torch.div(L, 16, rounding_mode='trunc')) 
            if len(buckets[L]) == 0:  buckets[L] = []
            buckets[L].append(idx)
            
            if len(buckets[L]) == self.batch_size:
                batch = list(buckets[L])
                yield batch
                yielded += 1
                buckets[L] = []
                
        batch = []
        leftover = [idx for bucket in buckets for idx in bucket]

        for idx in leftover:
            batch.append(idx)
            if len(batch) == self.batch_size:
                yielded += 1
                yield batch
                batch = []

        if len(batch) > 0 and not self.drop_last:
            yielded += 1
            yield batch
            
def dict_to(x, device='cuda'):
    return {k:x[k].to(device) for k in x}

def to_device(x, device='cuda'):
    return tuple(dict_to(e,device) for e in x)

class DeviceDataLoader:
    def __init__(self, dataloader, device='cuda'):
        self.dataloader = dataloader
        self.device = device
    
    def __len__(self):
        return len(self.dataloader)
    
    def __iter__(self):
        for batch in self.dataloader:
            yield tuple(dict_to(x, self.device) for x in batch)

In [5]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

In [6]:
class SinusoidalPosEmb(nn.Module):
    def __init__(self, dim=16, M=10000):
        super().__init__()
        self.dim = dim
        self.M = M

    def forward(self, x):
        device = x.device
        half_dim = self.dim // 2
        emb = math.log(self.M) / half_dim
        emb = torch.exp(torch.arange(half_dim, device=device) * (-emb))
        emb = x[...,None] * emb[None,...]
        emb = torch.cat((emb.sin(), emb.cos()), dim=-1)
        return emb

class RNA_Model(nn.Module):
    def __init__(self, dim=192, depth=12, head_size=32, **kwargs):
        super().__init__()
        self.emb = nn.Embedding(4,dim)
        self.pos_enc = SinusoidalPosEmb(dim)
        self.transformer = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=dim, nhead=dim//head_size, dim_feedforward=4*dim,
                dropout=0.1, activation=nn.GELU(), batch_first=True, norm_first=True), depth)
        self.proj_out = nn.Linear(dim,2)
    
    def forward(self, x0):
        mask = x0['mask']
        Lmax = mask.sum(-1).max()
        mask = mask[:,:Lmax]
        x = x0['seq'][:,:Lmax]
        
        pos = torch.arange(Lmax, device=x.device).unsqueeze(0)
        pos = self.pos_enc(pos)
        x = self.emb(x)
        x = x + pos
        
        x = self.transformer(x, src_key_padding_mask=~mask)
        x = self.proj_out(x)
        
        return x

In [7]:
def loss(pred,target):
    p = pred[target['mask'][:,:pred.shape[1]]]
    y = target['react'][target['mask']].clip(0,1)
    loss = F.l1_loss(p, y, reduction='none')
    loss = loss[~torch.isnan(loss)].mean()
    
    return loss

class MAE(Metric):
    def __init__(self): 
        self.reset()
        
    def reset(self): 
        self.x,self.y = [],[]
        
    def accumulate(self, learn):
        x = learn.pred[learn.y['mask'][:,:learn.pred.shape[1]]]
        y = learn.y['react'][learn.y['mask']].clip(0,1)
        self.x.append(x)
        self.y.append(y)

    @property
    def value(self):
        x,y = torch.cat(self.x,0),torch.cat(self.y,0)
        loss = F.l1_loss(x, y, reduction='none')
        loss = loss[~torch.isnan(loss)].mean()
        return loss

In [8]:
class RNA_Dataset_Test(Dataset):
    def __init__(self, df, mask_only=False, **kwargs):
        self.seq_map = {'A':0,'C':1,'G':2,'U':3}
        df['L'] = df.sequence.apply(len)
        self.Lmax = df['L'].max()
        self.df = df
        self.mask_only = mask_only
        
    def __len__(self):
        return len(self.df)  
    
    def __getitem__(self, idx):
        id_min, id_max, seq = self.df.loc[idx, ['id_min','id_max','sequence']]
        mask = torch.zeros(self.Lmax, dtype=torch.bool)
        L = len(seq)
        mask[:L] = True
        if self.mask_only: return {'mask':mask},{}
        ids = np.arange(id_min,id_max+1)
        
        seq = [self.seq_map[s] for s in seq]
        seq = np.array(seq)
        seq = np.pad(seq,(0,self.Lmax-L))
        ids = np.pad(ids,(0,self.Lmax-L), constant_values=-1)
        
        return {'seq':torch.from_numpy(seq), 'mask':mask}, \
               {'ids':ids}

In [10]:
class RNA_Dataset(Dataset):
    def __init__(self, df, mode='train', seed=2023, fold=0, nfolds=4, 
                 mask_only=False, sn = 1,**kwargs):
        self.seq_map = {'A':0,'C':1,'G':2,'U':3}
        self.Lmax = 206
        df['L'] = df.sequence.apply(len)
        df_2A3 = df.loc[df.experiment_type=='2A3_MaP']
        df_DMS = df.loc[df.experiment_type=='DMS_MaP']
        
        split = list(KFold(n_splits=nfolds, random_state=seed, 
                shuffle=True).split(df_2A3))[fold][0 if mode=='train' else 1]
        df_2A3 = df_2A3.iloc[split].reset_index(drop=True)
        df_DMS = df_DMS.iloc[split].reset_index(drop=True)
        
        if sn is not None:
            m = (df_2A3['signal_to_noise'].values >= sn) & (df_DMS['signal_to_noise'].values >= sn)
            df_2A3 = df_2A3.loc[m].reset_index(drop=True)
            df_DMS = df_DMS.loc[m].reset_index(drop=True)
        
        self.seq = df_2A3['sequence'].values
        self.L = df_2A3['L'].values
        
        self.react_2A3 = df_2A3[[c for c in df_2A3.columns if \
                                 'reactivity_0' in c]].values
        self.react_DMS = df_DMS[[c for c in df_DMS.columns if \
                                 'reactivity_0' in c]].values
        # self.react_err_2A3 = df_2A3[[c for c in df_2A3.columns if \
        #                          'reactivity_error_0' in c]].values
        # self.react_err_DMS = df_DMS[[c for c in df_DMS.columns if \
        #                         'reactivity_error_0' in c]].values
        self.sn_2A3 = df_2A3['signal_to_noise'].values
        self.sn_DMS = df_DMS['signal_to_noise'].values
        self.mask_only = mask_only
        
    def __len__(self):
        return len(self.seq)  
    
    def __getitem__(self, idx):
        seq = self.seq[idx]
        if self.mask_only:
            mask = torch.zeros(self.Lmax, dtype=torch.bool)
            mask[:len(seq)] = True
            return {'mask':mask},{'mask':mask}
        seq = [self.seq_map[s] for s in seq]
        seq = np.array(seq)
        mask = torch.zeros(self.Lmax, dtype=torch.bool)
        mask[:len(seq)] = True
        seq = np.pad(seq,(0,self.Lmax-len(seq)))
        
        react = torch.from_numpy(np.stack([self.react_2A3[idx],
                                           self.react_DMS[idx]],-1))
        # react_err = torch.from_numpy(np.stack([self.react_err_2A3[idx],
        #                                        self.react_err_DMS[idx]],-1))
        sn = torch.FloatTensor([self.sn_2A3[idx],self.sn_DMS[idx]])
        
        return {'seq':torch.from_numpy(seq), 'mask':mask}, \
               {'react':react,
                # 'react_err':react_err,
                'sn':sn, 'mask':mask}

## Run

In [ ]:
fold=0
fname = 'example0'
PATH = './'
OUT = './'
bs = 256
num_workers = 8
SEED = 2023
nfolds = 5
device = 'cuda'
sn=1

In [11]:
df = pd.read_parquet('train_data.parquet')

In [13]:
df2.columns = ['sequence_id','reactivity_0001',
 'reactivity_0002',
 'reactivity_0003',
 'reactivity_0004',
 'reactivity_0005',
 'reactivity_0006',
 'reactivity_0007',
 'reactivity_0008',
 'reactivity_0009',
 'reactivity_0010',
 'reactivity_0011',
 'reactivity_0012',
 'reactivity_0013',
 'reactivity_0014',
 'reactivity_0015',
 'reactivity_0016',
 'reactivity_0017',
 'reactivity_0018',
 'reactivity_0019',
 'reactivity_0020',
 'reactivity_0021',
 'reactivity_0022',
 'reactivity_0023',
 'reactivity_0024',
 'reactivity_0025',
 'reactivity_0026',
 'reactivity_0027',
 'reactivity_0028',
 'reactivity_0029',
 'reactivity_0030',
 'reactivity_0031',
 'reactivity_0032',
 'reactivity_0033',
 'reactivity_0034',
 'reactivity_0035',
 'reactivity_0036',
 'reactivity_0037',
 'reactivity_0038',
 'reactivity_0039',
 'reactivity_0040',
 'reactivity_0041',
 'reactivity_0042',
 'reactivity_0043',
 'reactivity_0044',
 'reactivity_0045',
 'reactivity_0046',
 'reactivity_0047',
 'reactivity_0048',
 'reactivity_0049',
 'reactivity_0050',
 'reactivity_0051',
 'reactivity_0052',
 'reactivity_0053',
 'reactivity_0054',
 'reactivity_0055',
 'reactivity_0056',
 'reactivity_0057',
 'reactivity_0058',
 'reactivity_0059',
 'reactivity_0060',
 'reactivity_0061',
 'reactivity_0062',
 'reactivity_0063',
 'reactivity_0064',
 'reactivity_0065',
 'reactivity_0066',
 'reactivity_0067',
 'reactivity_0068',
 'reactivity_0069',
 'reactivity_0070',
 'reactivity_0071',
 'reactivity_0072',
 'reactivity_0073',
 'reactivity_0074',
 'reactivity_0075',
 'reactivity_0076',
 'reactivity_0077',
 'reactivity_0078',
 'reactivity_0079',
 'reactivity_0080',
 'reactivity_0081',
 'reactivity_0082',
 'reactivity_0083',
 'reactivity_0084',
 'reactivity_0085',
 'reactivity_0086',
 'reactivity_0087',
 'reactivity_0088',
 'reactivity_0089',
 'reactivity_0090',
 'reactivity_0091',
 'reactivity_0092',
 'reactivity_0093',
 'reactivity_0094',
 'reactivity_0095',
 'reactivity_0096',
 'reactivity_0097',
 'reactivity_0098',
 'reactivity_0099',
 'reactivity_0100',
 'reactivity_0101',
 'reactivity_0102',
 'reactivity_0103',
 'reactivity_0104',
 'reactivity_0105',
 'reactivity_0106',
 'reactivity_0107',
 'reactivity_0108',
 'reactivity_0109',
 'reactivity_0110',
 'reactivity_0111',
 'reactivity_0112',
 'reactivity_0113',
 'reactivity_0114',
 'reactivity_0115',
 'reactivity_0116',
 'reactivity_0117',
 'reactivity_0118',
 'reactivity_0119',
 'reactivity_0120',
 'reactivity_0121',
 'reactivity_0122',
 'reactivity_0123',
 'reactivity_0124',
 'reactivity_0125',
 'reactivity_0126',
 'reactivity_0127',
 'reactivity_0128',
 'reactivity_0129',
 'reactivity_0130',
 'reactivity_0131',
 'reactivity_0132',
 'reactivity_0133',
 'reactivity_0134',
 'reactivity_0135',
 'reactivity_0136',
 'reactivity_0137',
 'reactivity_0138',
 'reactivity_0139',
 'reactivity_0140',
 'reactivity_0141',
 'reactivity_0142',
 'reactivity_0143',
 'reactivity_0144',
 'reactivity_0145',
 'reactivity_0146',
 'reactivity_0147',
 'reactivity_0148',
 'reactivity_0149',
 'reactivity_0150',
 'reactivity_0151',
 'reactivity_0152',
 'reactivity_0153',
 'reactivity_0154',
 'reactivity_0155',
 'reactivity_0156',
 'reactivity_0157',
 'reactivity_0158',
 'reactivity_0159',
 'reactivity_0160',
 'reactivity_0161',
 'reactivity_0162',
 'reactivity_0163',
 'reactivity_0164',
 'reactivity_0165',
 'reactivity_0166',
 'reactivity_0167',
 'reactivity_0168',
 'reactivity_0169',
 'reactivity_0170',
'reactivity_0171',
 'reactivity_0172',
 'reactivity_0173',
 'reactivity_0174',
 'reactivity_0175',
 'reactivity_0176',
 'reactivity_0177',
 'experiment_type',
 'signal_to_noise',
 'sequence']

In [16]:
seed_everything(SEED)

In [17]:
ds_train = RNA_Dataset(df, mode='train', fold=fold, nfolds=nfolds,sn=sn)


In [18]:
ds_train_len = RNA_Dataset(df, mode='train', fold=fold, 
            nfolds=nfolds, mask_only=True,sn=sn)

In [19]:
len(ds_train),len(ds_train_len)

(413374, 413374)

In [20]:
sampler_train = torch.utils.data.RandomSampler(ds_train_len) # randomly select samples

len_sampler_train = LenMatchBatchSampler(sampler_train, batch_size=bs,
            drop_last=True)

dl_train = DeviceDataLoader(torch.utils.data.DataLoader(ds_train, 
            batch_sampler=len_sampler_train, num_workers=num_workers,
            persistent_workers=True), device)

ds_val = RNA_Dataset(df, mode='eval', fold=fold, nfolds=nfolds)
ds_val_len = RNA_Dataset(df, mode='eval', fold=fold, nfolds=nfolds, 
           mask_only=True)

sampler_val = torch.utils.data.SequentialSampler(ds_val_len)
len_sampler_val = LenMatchBatchSampler(sampler_val, batch_size=bs, 
           drop_last=False)

dl_val= DeviceDataLoader(torch.utils.data.DataLoader(ds_val, 
           batch_sampler=len_sampler_val, num_workers=num_workers), device)
gc.collect()

0

In [21]:
data = DataLoaders(dl_train,dl_val)
model = RNA_Model()   
model = model.to(device)
# model.load_state_dict(torch.load('example0_0.pth',map_location=torch.device('cuda')))
learn = Learner(data, model, loss_func=loss,cbs=[GradientClip(3.0)],
            metrics=[MAE()]).to_fp16()

In [22]:
# learn.lr_find()

In [23]:
learn.fit_one_cycle(10, lr_max=1.4454397387453355e-05, pct_start=0.02)
torch.save(learn.model.state_dict(),os.path.join(OUT,f'model2.pth'))
gc.collect()

epoch,train_loss,valid_loss,mae,time
0,0.218330,0.234415,0.234866,05:01
1,0.201503,0.232055,0.232559,04:58
2,0.193914,0.231752,0.232275,04:59
3,0.190076,0.231313,0.231865,05:00
4,0.187435,0.230562,0.231099,04:59
5,0.186071,0.229915,0.230462,05:00


KeyboardInterrupt: 

In [24]:
torch.save(learn.model.state_dict(),os.path.join(OUT,f'model2.pth'))
gc.collect()

2938

## Inference

In [25]:
df_test = pd.read_parquet('test_sequences.parquet')

In [26]:
ds = RNA_Dataset_Test(df_test)
dl = DeviceDataLoader(torch.utils.data.DataLoader(ds, batch_size=bs, 
               shuffle=False, drop_last=False, num_workers=num_workers), device)

In [27]:
del df_test
gc.collect()

0

In [28]:
model = RNA_Model()   
model = model.to(device)
# model.load_state_dict(torch.load('model_fintune.pth',map_location=torch.device('cpu')))
model.load_state_dict(torch.load('model2.pth',map_location=torch.device('cpu')))
model.eval()

RNA_Model(
  (emb): Embedding(4, 192)
  (pos_enc): SinusoidalPosEmb()
  (transformer): TransformerEncoder(
    (layers): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=192, out_features=192, bias=True)
        )
        (linear1): Linear(in_features=192, out_features=768, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=768, out_features=192, bias=True)
        (norm1): LayerNorm((192,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((192,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
        (activation): GELU(approximate=none)
      )
      (1): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=192, out_features=192, bias=True)
        )
     

In [30]:
import gc
gc.collect()

9

In [ ]:
ids,preds = [],[]
for x,y in tqdm(dl):
    with torch.no_grad(),torch.cuda.amp.autocast():
        # p = torch.stack([torch.nan_to_num(model(x)) for model in models]
        #                 ,0).mean(0)
        p = torch.nan_to_num(model(x))
        
    for idx, mask, pi in zip(y['ids'].cpu(), x['mask'].cpu(), p.cpu()):
        ids.append(idx[mask])
        preds.append(pi[mask[:pi.shape[0]]])



100%|█████████▉| 5235/5250 [05:29<00:02,  5.13it/s]

In [ ]:
ids = torch.concat(ids)
preds = torch.concat(preds)

In [ ]:
df = pd.DataFrame({'id':ids.numpy(), 'reactivity_DMS_MaP':preds[:,1].numpy(), 
                   'reactivity_2A3_MaP':preds[:,0].numpy()})
# df.to_csv('submission.csv', index=False, float_format='%.4f') # 6.5GB

In [ ]:
df

In [ ]:
df['reactivity_DMS_MaP']=df['reactivity_DMS_MaP'].astype(float)
df['reactivity_2A3_MaP']=df['reactivity_2A3_MaP'].astype(float)

In [ ]:
df.to_parquet('submission.parquet') # 6.5GB